In [11]:
import os
import glob

exp_path = "/research/d5/gds/yzhong22/experiments/med_vlm_benchmark"
exp_path_mirror = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark"

In [12]:
# train

tune_modules = ["L", "M", "ML", "VLM", "VM", "V"]
tune_modules = [
    # "V",
    "VLM",
    "VM",
    "V",
]

# datasets = {"SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs", "PathVQA": "None", "VQA-RAD": "None"}

datasets = {
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k"}

base_command = """deepspeed --master_port 29600 run_train.py \
    --peft lora --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed ./script/zero3.json \
    --task vqa --dataset {} \
    --model LLaVA-1.5 --version v1 \
    --image_path {} \
    --model_path /research/d5/gds/yzhong22/misc/pretrained/llava-v1.5-7b \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --num_train_epochs 3 \
    --per_device_train_batch_size {} \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps {} \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing {} \
    --dataloader_num_workers 4 \
    --tune_modules {}"""

per_device_train_batch_size = 16
gradient_accumulation_steps = 1
gradient_checkpointing = True

bs_scale_factor = 8

assert per_device_train_batch_size * gradient_accumulation_steps == 16

commands = []

for data, path in datasets.items():
    for tm in tune_modules:
        if "V" not in tm:
            commands.append(
                base_command.format(
                    data, path, per_device_train_batch_size, gradient_accumulation_steps, gradient_checkpointing, tm
                )
            )
        else:
            commands.append(
                base_command.format(
                    data,
                    path,
                    per_device_train_batch_size // bs_scale_factor,
                    int(gradient_accumulation_steps * bs_scale_factor),
                    False,
                    tm,
                )
            )

print("&&".join(commands))

deepspeed --master_port 29600 run_train.py     --peft lora --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5     --deepspeed ./script/zero3.json     --task vqa --dataset Harvard-FairVLMed10k     --model LLaVA-1.5 --version v1     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model_path /research/d5/gds/yzhong22/misc/pretrained/llava-v1.5-7b     --mm_projector_type mlp2x_gelu     --mm_vision_select_layer -2     --mm_use_im_start_end False     --mm_use_im_patch_token False     --image_aspect_ratio pad     --group_by_modality_length True     --bf16 True     --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --num_train_epochs 3     --per_device_train_batch_size 2     --per_device_eval_batch_size 4     --gradient_accumulation_steps 8     --evaluation_strategy "no"     --save_strategy "steps"     --save_steps 50000     --save_total_limit 1     --learning_rate 2e-5     --weight_decay 0.

# EVAL

## zero-shot

In [2]:
# qwen2

datasets = {
    # "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model Qwen2-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2-VL-7B-Instruct \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path))

print("&&".join(commands))

# print(commands[3])

python run_eval.py     --task vqa --dataset PathVQA --split test     --image_path None     --model Qwen2-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset VQA-RAD --split test     --image_path None     --model Qwen2-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset Harvard-FairVLMed10k --split test     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model Qwen2-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gd

In [7]:
# qwen25

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    # "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model Qwen25-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2.5-VL-7B-Instruct \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path))

print("&&".join(commands))

python run_eval.py     --task vqa --dataset SLAKE --split test     --image_path /research/d5/gds/yzhong22/datasets/SLAKE/imgs     --model Qwen25-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2.5-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset VQA-RAD --split test     --image_path None     --model Qwen25-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2.5-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset Harvard-FairVLMed10k --split test     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model Qwen25-VL --model_path /research/d5/gds/yzhong22/misc/pretrained/Qwen2.5-VL-7B-Instruct     --exp_path /research/d5/gds/yzhong22/experiments/m

In [ ]:
# zero-shot eval